In [2]:
# Import VoidX configuration
import voidx
from voidx import *

# Enable LaTeX for text rendering
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 14  # Adjust font size

# Configuration

In [ ]:
from config import get_config

config = get_config()  # picks up everything from config/global.json

# For backward compatibility, create variables from config
box = config.box
local = config.local
hdf = config.hdf
VIDE = config.VIDE
name = config.name
fraction_in_voids = config.fraction_in_voids
model_name = config.model_name
device = config.device
seed = config.seed

# Paths are automatically set up
data_dir = config.data_dir
checkpoint_dir_spec = config.checkpoint_dir_spec
checkpoint_dir_global = config.checkpoint_dir_global
plot_dir = config.plot_dir
result_dir = config.result_dir

# Display configuration
config.print_info()

# Load voids and galaxies

In [ ]:
if VIDE : 
    load = False  # Load particles in voids
    void_dir = 'sample_VIDE'
    void_dir = os.path.join(data_dir, void_dir)
    void_catalogue = vide.loadVoidCatalog(void_dir, dataPortion='all', untrimmed=True, loadParticles=load)
    vide_radius = vide.getArray(void_catalogue.voids, 'radius')
    vide_redshift = vide.getArray(void_catalogue.voids, 'redshift')
    vide_ra = vide.getArray(void_catalogue.voids, 'RA')
    vide_dec = vide.getArray(void_catalogue.voids, 'Dec')
    vide_macrocenter = vide.getArray(void_catalogue.voids, 'macrocenter')
    vide_ellipticity = vide.getArray(void_catalogue.voids, 'ellipticity')
    vide_voidVol = vide.getArray(void_catalogue.voids, 'voidVol')
    vide_densCon = vide.getArray(void_catalogue.voids, 'densCon')
    vide_parentID = vide.getArray(void_catalogue.voids, 'parentID')
    vide_numChildren = vide.getArray(void_catalogue.voids, 'numChildren')
    vide_centralDen = vide.getArray(void_catalogue.voids, 'centralDen')
    vide_coreDens = vide.getArray(void_catalogue.voids, 'coreDens')

    vide_ID = [0]*len(void_catalogue.voids)
    for i in range(len(void_catalogue.voids)):
        vide_ID[i] = void_catalogue.voids[i].voidID
    vide_ID = np.array(vide_ID)

    if load:
        # Count number of particles in voids
        vide_part = np.zeros(len(vide_radius))
        for i, void in tqdm(enumerate(vide_ID), total=len(vide_ID)):
            voidPart = vide.getVoidPart(void_catalogue, int(void))
            vide_part[i] = len(voidPart)
        print('Total number of particles in voids:', np.sum(vide_part))

In [ ]:
if not VIDE : 
    npzfile = np.load(os.path.join(data_dir, f"voronoi.npz"))
    voronoi_radii = npzfile['voronoi_radii']
    print('Number of voids:', len(voronoi_radii))   
    print('Range of radii :', np.min(voronoi_radii), np.max(voronoi_radii))
    voronoi_redshifts = npzfile['voronoi_redshifts']
    voronoi_voidID = npzfile['voronoi_voidID']
    voronoi_ra = npzfile['voronoi_RA']
    voronoi_dec = npzfile['voronoi_Dec']
    print('Range of RA :', np.min(voronoi_ra), np.max(voronoi_ra))
    print('Range of Dec :', np.min(voronoi_dec), np.max(voronoi_dec)) 

# Load galaxies 

### If hdf5 file

In [ ]:
if hdf:
    with h5py.File(data_dir / 'galaxies.hdf5', 'r') as f:
        print("Keys:", list(f.keys()))

        galaxy_mass = f['mass'][:]
        galaxy_pos = f['pos'][:]
        galaxy_vel = f['vel'][:]

    galaxy_mass = galaxy_mass.squeeze() 
    galaxy_pos = np.reshape(galaxy_pos, (len(galaxy_pos), 3))
    galaxy_vel = np.reshape(galaxy_vel, (len(galaxy_vel), 3))

    print('Shape of galaxy mass array:', galaxy_mass.shape)
    print('Shape of galaxy position array:', galaxy_pos.shape)
    print('Shape of galaxy velocity array:', galaxy_vel.shape)

### If fits file

In [ ]:
if not hdf:
    galaxy_fits_file = data_dir / 'galaxy.fits'
    with fits.open(galaxy_fits_file) as hdul:
        galaxy_data = hdul[1].data
        print(galaxy_data.columns)
        galaxy_ra = galaxy_data['ra']
        galaxy_dec = galaxy_data['dec']
        galaxy_z = galaxy_data['redshift']
        print('Number of galaxies:', len(galaxy_ra))
        print('Range of redshifts :', np.min(galaxy_z), np.max(galaxy_z))
        print('Range of RA :', np.min(galaxy_ra), np.max(galaxy_ra))
        print('Range of Dec :', np.min(galaxy_dec), np.max(galaxy_dec))

        ra_shift = 33.75
        galaxy_ra = galaxy_ra + ra_shift
        print('After RA shift')
        print('Range of RA :', np.min(galaxy_ra), np.max(galaxy_ra))

        if VIDE : 
            if np.min(galaxy_ra) > np.max(vide_ra) or np.max(galaxy_ra) < np.min(vide_ra):
                raise ValueError("Galaxy positions are shifted in RA compared to void positions.")
        else : 
            if np.min(galaxy_ra) > np.max(voronoi_ra) or np.max(galaxy_ra) < np.min(voronoi_ra):
                raise ValueError("Galaxy positions are shifted in RA compared to void positions.")
        

# Clean voids

In [ ]:
if VIDE :     
    radius_min, radius_max = 10, 100
    part_min, part_max = 10, 1e5

    mask = (vide_radius >= radius_min) & (vide_radius <= radius_max) 
    if load : 
        mask_part =  (vide_part >= part_min) & (vide_part <= part_max)
    else: 
        mask_part = np.ones(len(vide_radius), dtype=bool)
    mask = mask & mask_part

    vide_radius = vide_radius[mask]
    vide_redshift = vide_redshift[mask]
    vide_ra = vide_ra[mask]
    vide_dec = vide_dec[mask]
    vide_macrocenter = vide_macrocenter[mask]
    vide_ellipticity = vide_ellipticity[mask]
    if load : vide_part = vide_part[mask]
    vide_ID = vide_ID[mask]

    print('We are left with', len(vide_radius), 'voids after applying the cuts.')

In [ ]:
if not VIDE : 
    radius_min, radius_max = 10, 100
    mask = (voronoi_radii >= radius_min) & (voronoi_radii <= radius_max)

    voronoi_radii = voronoi_radii[mask]
    voronoi_redshifts = voronoi_redshifts[mask]
    voronoi_ra = voronoi_ra[mask]
    voronoi_dec = voronoi_dec[mask]
    voronoi_voidID = voronoi_voidID[mask]

    print('We are left with', len(voronoi_radii), 'voids after applying the cuts.')

# 3D plots of galaxy distribution and voids

## If box 

### 3D plot of voids

In [ ]:
if box : 
    ratio_box = 4  # side length of inner box is 1/ratio_box of full box

    x_min, x_max = vide_macrocenter[:, 0].min(), vide_macrocenter[:, 0].max()
    y_min, y_max = vide_macrocenter[:, 1].min(), vide_macrocenter[:, 1].max()
    z_min, z_max = vide_macrocenter[:, 2].min(), vide_macrocenter[:, 2].max()

    Lx, Ly, Lz = (x_max - x_min), (y_max - y_min), (z_max - z_min)
    cx, cy, cz = (x_min + x_max)/2, (y_min + y_max)/2, (z_min + z_max)/2
    hx, hy, hz = Lx / (2 * ratio_box), Ly / (2 * ratio_box), Lz / (2 * ratio_box)

    mask = (
        (vide_macrocenter[:, 0] >= cx - hx) & (vide_macrocenter[:, 0] <= cx + hx) &
        (vide_macrocenter[:, 1] >= cy - hy) & (vide_macrocenter[:, 1] <= cy + hy) &
        (vide_macrocenter[:, 2] >= cz - hz) & (vide_macrocenter[:, 2] <= cz + hz)
    )

    x_voids = vide_macrocenter[mask, 0]
    y_voids = vide_macrocenter[mask, 1]
    z_voids = vide_macrocenter[mask, 2]
    centers_voids = vide_macrocenter[mask, :]
    radii_voids = vide_radius[mask]
    ellipticity_voids = vide_ellipticity[mask]
    redshift_voids = vide_redshift[mask]
    # if Load : part_voids = vide_part[mask]
    ID_voids = vide_ID[mask]
    print('Number of voids in this region:', len(x_voids))

    # Create 3D scatter plot for voids
    fig_voids = go.Figure(data=[go.Scatter3d(
        x=x_voids,
        y=y_voids,
        z=z_voids,
        mode='markers',
        marker=dict(
            size=radii_voids / 2,  # Scale down the size for better visualization
            color=radii_voids,  # Color by radius
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Void Radius [Mpc/h]')
        ),
    )])
    # Update layout
    fig_voids.update_layout(
        title=f'Void Distribution (1/{ratio_box**3} of Box Volume - Central Region)',
        scene=dict(
            xaxis_title='X [Mpc/h]',
            yaxis_title='Y [Mpc/h]',
            zaxis_title='Z [Mpc/h]',
            aspectmode='cube'  # Ensures the plot has equal scaling for all axes
        ),
        width=1000,
        height=800
    )
    fig_voids.show()

In [ ]:
if box : 

    x_min, x_max = vide_macrocenter[:, 0].min(), vide_macrocenter[:, 0].max()
    y_min, y_max = vide_macrocenter[:, 1].min(), vide_macrocenter[:, 1].max()
    z_min, z_max = vide_macrocenter[:, 2].min(), vide_macrocenter[:, 2].max()

    Lx, Ly, Lz = (x_max - x_min), (y_max - y_min), (z_max - z_min)
    cx, cy, cz = (x_min + x_max)/2, (y_min + y_max)/2, (z_min + z_max)/2
    hx, hy, hz = Lx / (2 * ratio_box), Ly / (2 * ratio_box), Lz / (2 * ratio_box)

    mask = (
        (vide_macrocenter[:, 0] >= cx - hx) & (vide_macrocenter[:, 0] <= cx + hx) &
        (vide_macrocenter[:, 1] >= cy - hy) & (vide_macrocenter[:, 1] <= cy + hy) &
        (vide_macrocenter[:, 2] >= cz - hz) & (vide_macrocenter[:, 2] <= cz + hz)
    )

    x_voids = vide_macrocenter[mask, 0]
    y_voids = vide_macrocenter[mask, 1]
    z_voids = vide_macrocenter[mask, 2]
    centers_voids = vide_macrocenter[mask, :]
    radii_voids = vide_radius[mask]
    ellipticity_voids = vide_ellipticity[mask]
    redshift_voids = vide_redshift[mask]
    # if Load : part_voids = vide_part[mask]
    ID_voids = vide_ID[mask]
    print('Number of voids in this region:', len(x_voids))   

    # Create 3D plot with voids as spheres with correct physical size
    fig_voids = go.Figure()

    scaling_rad = 0.8
    
    # Add each void as a sphere with its physical radius
    for i in range(len(x_voids)):
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = scaling_rad * radii_voids[i] * np.cos(u) * np.sin(v) + x_voids[i]
        y = scaling_rad * radii_voids[i] * np.sin(u) * np.sin(v) + y_voids[i]
        z = scaling_rad * radii_voids[i] * np.cos(v) + z_voids[i]
        
        fig_voids.add_trace(go.Surface(
            x=x, y=y, z=z,
            opacity=0.3,
            showscale=False,
            colorscale='Viridis',
            surfacecolor=np.full_like(x, scaling_rad * radii_voids[i]),  # Color by radius
            name=f'Void {ID_voids[i]} (R={scaling_rad * radii_voids[i]:.1f})'
        ))
    
    # Update layout (keep this part)
    fig_voids.update_layout(
        title=f'Void Distribution (1/{ratio_box**3} of Box Volume - Central Region)',
        scene=dict(
            xaxis_title='X [Mpc/h]',
            yaxis_title='Y [Mpc/h]',
            zaxis_title='Z [Mpc/h]',
            aspectmode='cube'
        ),
        width=1000,
        height=800
    )
    fig_voids.show()

### 3D plots of galaxies 

In [ ]:
if box : 
    # --- Inner box selection for galaxies ---
    mask_inner_galaxies = (
        (galaxy_pos[:, 0] >= cx - hx) & (galaxy_pos[:, 0] <= cx + hx) &
        (galaxy_pos[:, 1] >= cy - hy) & (galaxy_pos[:, 1] <= cy + hy) &
        (galaxy_pos[:, 2] >= cz - hz) & (galaxy_pos[:, 2] <= cz + hz)
    )
    x_galaxy = galaxy_pos[mask_inner_galaxies, 0]
    y_galaxy = galaxy_pos[mask_inner_galaxies, 1]
    z_galaxy = galaxy_pos[mask_inner_galaxies, 2]
    mass_galaxy = galaxy_mass[mask_inner_galaxies]
    vel_galaxy = galaxy_vel[mask_inner_galaxies, :]
    print('We are left with', len(x_galaxy), 'galaxies in the inner box.')
    print('Range of galaxy positions:', x_galaxy.min(), x_galaxy.max(), y_galaxy.min(), y_galaxy.max(), z_galaxy.min(), z_galaxy.max())

    # Create 3D scatter plot for galaxies

    fig_galaxy = go.Figure(data=[go.Scatter3d(
        x=x_galaxy,
        y=y_galaxy,
        z=z_galaxy,
        mode='markers',
        marker=dict(
            size=2,
            color=mass_galaxy,  # Color by mass
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Mass')
        )
    )])

    # Update layout
    fig_galaxy.update_layout(
        title='Galaxy Distribution (1/64th of Box Volume - Spatial Corner)',
        scene=dict(
            xaxis_title='X [Mpc/h]',
            yaxis_title='Y [Mpc/h]',
            zaxis_title='Z [Mpc/h]'
        ),
        width=1000,
        height=800
    )
    fig_galaxy.show()


### Both plots

In [ ]:
if box :     
    # --- 3D plot all the voids and their galaxies ---

    # Change size of figure
    fig = go.Figure(layout=dict(width=800, height=800))
    # Add voids as spheres
    for i in range(len(x_voids)):
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = radii_voids[i] * np.cos(u) * np.sin(v) + x_voids[i]
        y = radii_voids[i] * np.sin(u) * np.sin(v) + y_voids[i]
        z = radii_voids[i] * np.cos(v) + z_voids[i]
        fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.3, showscale=False, name=f'Void {i}'))
    # Add galaxies as points
    fig.add_trace(go.Scatter3d(x=x_galaxy, y=y_galaxy, z=z_galaxy,
                                mode='markers', marker=dict(size=2, color='blue'), name='Galaxies'))
    # Update layout
    fig.update_layout(title='Voids and Galaxies in Inner Box',
                    scene=dict(xaxis_title='X [Mpc/h]', yaxis_title='Y [Mpc/h]', zaxis_title='Z [Mpc/h]'),
                    showlegend=True)
    fig.show()


## If lightcone

### 3D plot of voids

In [ ]:
if not box : 
    if VIDE :
        ra_center = vide_ra.mean()  # degrees
        dec_center = vide_dec.mean()  # degrees
        redshift_center = vide_redshift.mean()
    else :
        ra_center = voronoi_ra.mean()
        dec_center = voronoi_dec.mean()
        redshift_center = voronoi_redshifts.mean()
    delta_ra = 3.0  # degrees
    delta_dec = 3.0  # degrees
    delta_redshift = 0.08 # redshift range
    ra_min = ra_center - delta_ra / 2
    ra_max = ra_center + delta_ra / 2
    dec_min = dec_center - delta_dec / 2
    dec_max = dec_center + delta_dec / 2
    redshift_min = redshift_center - delta_redshift / 2
    redshift_max = redshift_center + delta_redshift / 2

    print('RA range:', ra_min, ra_max)
    print('Dec range:', dec_min, dec_max)
    print('Redshift range:', redshift_min, redshift_max)

    if VIDE : 
        sky_mask_voids = ((vide_ra >= ra_min) & (vide_ra <= ra_max) &
                    (vide_dec >= dec_min) & (vide_dec <= dec_max) &
                    (vide_redshift >= redshift_min) & (vide_redshift <= redshift_max))
        ra_voids = vide_ra[sky_mask_voids]
        dec_voids = vide_dec[sky_mask_voids]
        redshift_voids = vide_redshift[sky_mask_voids]
        radii_voids = vide_radius[sky_mask_voids]
        ID_voids = vide_ID[sky_mask_voids]
    
    else : 
        sky_mask_voids = ((voronoi_ra >= ra_min) & (voronoi_ra <= ra_max) &
                    (voronoi_dec >= dec_min) & (voronoi_dec <= dec_max) &
                    (voronoi_redshifts >= redshift_min) & (voronoi_redshifts <= redshift_max))
        ra_voids = voronoi_ra[sky_mask_voids]
        dec_voids = voronoi_dec[sky_mask_voids]
        redshift_voids = voronoi_redshifts[sky_mask_voids]
        radii_voids = voronoi_radii[sky_mask_voids]
        ID_voids = voronoi_voidID[sky_mask_voids]
    
    print('Number of voids in this sky patch:', len(ra_voids))

    x_voids = convert_to_Cartesian(ra_voids, dec_voids, redshift_voids)[0]
    y_voids = convert_to_Cartesian(ra_voids, dec_voids, redshift_voids)[1]
    z_voids = convert_to_Cartesian(ra_voids, dec_voids, redshift_voids)[2]
    centers_voids = np.column_stack((x_voids, y_voids, z_voids))

    print('Range of void Cartesian coordinates:', x_voids.min(), x_voids.max(), y_voids.min(), y_voids.max(), z_voids.min(), z_voids.max())

    fig_voids_cartesian = go.Figure(data=[go.Scatter3d(
        x=x_voids,
        y=y_voids,
        z=z_voids,
        mode='markers',
        marker=dict(
            size=radii_voids * 2,  # Scale size for better visualization
            color=radii_voids,  # Color by radius
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Void Radius [Mpc/h]')
        )
    )])

    fig_voids_cartesian.update_layout(
        title='Void Distribution (Cartesian Coordinates)',
        scene=dict(
            xaxis_title='X [Mpc/h]',
            yaxis_title='Y [Mpc/h]',
            zaxis_title='Z [Mpc/h]',
            aspectmode='cube'  # Ensures the plot has equal scaling for all axes
        ),
        width=1000,
        height=800
    )
    fig_voids_cartesian.show()

### 3D plots of galaxies 

In [ ]:
if not box : 

    sky_mask = ((galaxy_ra >= ra_min) & (galaxy_ra <= ra_max) &
                (galaxy_dec >= dec_min) & (galaxy_dec <= dec_max) &
                (galaxy_z >= redshift_min) & (galaxy_z <= redshift_max))
    ra_galaxy = galaxy_ra[sky_mask]
    dec_galaxy = galaxy_dec[sky_mask]
    redshift_galaxy = galaxy_z[sky_mask]

    print('Number of galaxies in this sky patch:', len(ra_galaxy))

    x_galaxy = convert_to_Cartesian(ra_galaxy, dec_galaxy, redshift_galaxy)[0]
    y_galaxy = convert_to_Cartesian(ra_galaxy, dec_galaxy, redshift_galaxy)[1]
    z_galaxy = convert_to_Cartesian(ra_galaxy, dec_galaxy, redshift_galaxy)[2]

    # Create 3D scatter plot for galaxies in Cartesian coordinates
    fig_galaxy_cartesian = go.Figure(data=[go.Scatter3d(
        x=x_galaxy,
        y=y_galaxy,
        z=z_galaxy,
        mode='markers',
        marker=dict(
            size=1,
            color='darkblue',  # Uniform color for galaxies
            opacity=0.8
        )
    )])
    # Update layout
    fig_galaxy_cartesian.update_layout(
        title='Galaxy Distribution (Cartesian Coordinates)',
        scene=dict(
            xaxis_title='X [Mpc/h]',
            yaxis_title='Y [Mpc/h]',
            zaxis_title='Z [Mpc/h]'
        ),
        width=1000,
        height=800
    )
    fig_galaxy_cartesian.show()
    

### Plot voids and particles in the same 3D plot

In [ ]:
if not box :      
    fig = go.Figure(layout=dict(width=800, height=800))
    # Add voids as spheres
    for i in range(len(x_voids)):
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = radii_voids[i] * np.cos(u) * np.sin(v) + x_voids[i]
        y = radii_voids[i] * np.sin(u) * np.sin(v) + y_voids[i]
        z = radii_voids[i] * np.cos(v) + z_voids[i]
        fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.8, showscale=False, name=f'Void {i}'))
    # Add galaxies as points
    fig.add_trace(go.Scatter3d(x=x_galaxy, y=y_galaxy, z=z_galaxy,
                                mode='markers', marker=dict(size=0.5, color='blue'), name='Galaxies'))
    # Update layout
    fig.update_layout(title='Voids and Galaxies in Inner Box',
                    scene=dict(xaxis_title='X [Mpc/h]', yaxis_title='Y [Mpc/h]', zaxis_title='Z [Mpc/h]'),
                    showlegend=True)
    fig.show()


In [ ]:
if not box and VIDE :
    # Galaxies (inner-box)
    G = np.column_stack((x_galaxy, y_galaxy, z_galaxy))

    # IDs aligned with centers_voids/radii_voids (same order/length)
    void_ids_aligned = np.asarray(vide_ID[sky_mask_voids], dtype=int)
    id2idx = {int(v): i for i, v in enumerate(void_ids_aligned)}

    # Unique sorted list of void IDs to show
    void_ids_sorted = sorted(set(int(v) for v in void_ids_aligned))
    valid_voids = [vid for vid in void_ids_sorted if vid in id2idx]

    # Optional: subsample for "All" view
    max_points_all = 500000
    if len(G) > max_points_all:
        idx = np.random.RandomState(0).choice(len(G), size=max_points_all, replace=False)
        Gall = G[idx]
    else:
        Gall = G

    # Precompute sphere grid
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]

    traces = []
    trace_labels = []  # tuples to drive visibility
    counts_by_vid = {}

    # 0) All-galaxies trace
    traces.append(go.Scatter3d(
        x=Gall[:, 0], y=Gall[:, 1], z=Gall[:, 2],
        mode='markers',
        marker=dict(size=2, color='gray'),
        name='All galaxies (inner-box voids)'
    ))
    trace_labels.append(('all', None))

    # 1) One scatter + one sphere per void
    for vid in valid_voids:
        i = id2idx[vid]
        cx, cy, cz = centers_voids[i]
        R = float(radii_voids[i])
        Rsel = 1.3 * R

        # Select galaxies within 1.5*R from this void's center
        dx = G[:, 0] - cx
        dy = G[:, 1] - cy
        dz = G[:, 2] - cz
        msel = (dx*dx + dy*dy + dz*dz) <= (Rsel * Rsel)
        Gsel = G[msel]
        nsel = len(Gsel)
        counts_by_vid[vid] = nsel

        # Per-void galaxies
        traces.append(go.Scatter3d(
            x=Gsel[:, 0] if nsel else [],
            y=Gsel[:, 1] if nsel else [],
            z=Gsel[:, 2] if nsel else [],
            mode='markers',
            marker=dict(size=3),
            name=f'Void {vid} galaxies ({nsel})',
            showlegend=False
        ))
        trace_labels.append(('void', vid))

        # Per-void sphere
        xs = cx + R * np.cos(u) * np.sin(v)
        ys = cy + R * np.sin(u) * np.sin(v)
        zs = cz + R * np.cos(v)
        traces.append(go.Surface(
            x=xs, y=ys, z=zs,
            opacity=0.15,
            showscale=False,
            name=f'Void {vid} sphere',
            hoverinfo='skip'
        ))
        trace_labels.append(('void', vid))

    # Visibility helpers
    def vis_for_all():
        return [lbl[0] == 'all' for lbl in trace_labels]

    def vis_for_void(target_vid):
        tv = int(target_vid)
        return [lbl[0] == 'void' and lbl[1] == tv for lbl in trace_labels]

    # Build buttons
    buttons = []
    buttons.append(dict(
        label='All',
        method='update',
        args=[
            {'visible': vis_for_all()},
            {'title': f'All galaxies (inner-box voids): {len(G)} total'}
        ]
    ))

    for vid in valid_voids:
        n = counts_by_vid.get(int(vid), 0)
        R = float(radii_voids[id2idx[int(vid)]])
        buttons.append(dict(
            label=f'Void {vid} ({n})',
            method='update',
            args=[
                {'visible': vis_for_void(vid)},
                {'title': f'Void {vid}: {n} galaxies within 1.5R, R={R:.2f}'}
            ]
        ))

    # Initial visibility (show All)
    initial_visible = vis_for_all()

    fig = go.Figure(data=traces)
    fig.update_layout(
        updatemenus=[dict(type='dropdown', showactive=True, buttons=buttons, x=1.02, xanchor='left', y=1)],
        scene=dict(xaxis_title='X [Mpc/h]', yaxis_title='Y [Mpc/h]', zaxis_title='Z [Mpc/h]'),
        title='All galaxies (inner-box voids)',
        width=900, height=800
    )
    fig.update_traces(visible=False)
    for i, vis in enumerate(initial_visible):
        fig.data[i].visible = vis

    fig.show()